### step 4

In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:

# min_ = np.swapaxes(min_,0,1)
# mean = np.swapaxes(mean,0,1)
# range_ = np.swapaxes(range_,0,1)
# std = np.swapaxes(std,0,1)

In [4]:
def train_valid(X_train,Y_train,X_valid,Y_valid,fold):
    model = [resnet.ResnetBuilder.build_resnet_18((1, X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_34((1,X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_50((1, X_train.shape[1], X_train.shape[2]), 41) ,
             resnet.ResnetBuilder.build_resnet_101((1, X_train.shape[1], X_train.shape[2]), 41),]
#              resnet.ResnetBuilder.build_resnet_152((1, X_train.shape[1], X_train.shape[2]), 41)]
    kk = random.randint(0, 3)#4)
    model = model[kk]
    print('using resnet model: '+str(kk))

    model.summary()
    if kk>=3:
        batchSize=[32]
    elif kk>2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    if not os.path.exists(saveD):
        os.makedirs(saveD)
    opt = Adam()#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
        width_shift_range=0.1*random.random(),
        height_shift_range=0.1*random.random(),
        shear_range=0.15*random.random(),
        preprocessing_function=get_random_eraser(p=0.8+0.15*random.random(),v_l=np.min(X_train), v_h=np.max(X_train)) # Trainset's boundaries.
    )
#     datagen.fit(X_train)
    generator = MixupGenerator(X_train, Y_train, alpha=1, 
                               batch_size=batchSize, datagen=datagen)

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    print('using resnet model: '+str(kk))
    if not os.path.exists(logD):
        os.makedirs(logD)
    history = History()
    callback=[
    #     ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/2),
    #                                   min_lr=1e-4,mode='min', cooldown=1 ),
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_resnet'+str(kk)),
        history#,batch_size=batch_size, write_graph=True, write_grads=False, write_images=True)
    ]

    model.fit_generator(generator(),
                        steps_per_epoch=X_train.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch
                       )
    return model,kk

In [5]:
def get_semi_data(X_train,Y_train):
    X_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_un_ver.npy')
    X_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_test_ver.npy')
    X = np.concatenate((X_un_ver,X_test_ver))
    Y_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_un_ver.npy')
    Y_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_test_ver.npy')
    Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [6]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(model,X_semi,Y_semi,fold,kk):
    name = glob.glob('model/'+feature_type+'/'+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    if kk==4:
        batchSize=[32]
    elif kk>=2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
#     batchSize=[32,64,128,256] ##ERR?
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
#         featurewise_std_normalization=True,
        width_shift_range=0.05+0.15*random.random(),
        height_shift_range=0.05+0.15*random.random(),
        shear_range=0.084375+0.253125*random.random(),
        preprocessing_function=get_random_eraser(v_l=np.min(X_semi), v_h=np.max(X_semi)) # Trainset's boundaries.
    )
#     datagen.fit(X_semi)
    test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    generator = MixupGenerator(X_semi, Y_semi, alpha=0.4+0.6*random.random(), 
                               batch_size=batchSize, datagen=datagen)
    

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/10),min_lr=4e-6,
                          mode='min', cooldown=1,verbose=1 ), #0.2,/25
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_semi_fold'+str(fold)+'_resnet'+str(kk)+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_semi_resnet'+str(kk)),
        history
    ]
    model.fit_generator(generator(),
                        steps_per_epoch=X_semi.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch,
                        initial_epoch = int(patien/25)
                       )
#     model.fit(X_semi,Y_semi,
#               shuffle=True,
#               callbacks=callback, 
#               class_weight='auto',
#               validation_data=(X_valid,Y_valid),
#               batch_size=batchSize,
#               epochs=epoch)
    return model

In [7]:
feature_type = 'mfcc7'

In [8]:
# normalized parameters
mean = np.load('feature/'+feature_type+'/mean.npy')
std = np.load('feature/'+feature_type+'/std.npy')
min_ = np.load('feature/'+feature_type+'/min.npy')
range_ = np.load('feature/'+feature_type+'/range.npy')


base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [6,7,8,9]#str(sys.argv[1])

In [ ]:
# fold=0
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
    print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
    model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(model,X_semi,Y_semi,fold,model_num)
#0=>0.84367(semi)/0.83288(not semi)
#1=>0.84367(resnet3_semi)/0.81132(not semi)
#2=>0.84097(resnet2_semi)/0.81671(not semi)
#3=>0.88410(resnet1_semi)/0.84367
#4=>0.82210(resnet4_semi)/0.74663(not semi)
#5 => 0.82749(resnet4_semi)/0.77628
#6 => resnet2

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 60, 259, 1) (3339, 41)
===train verified_fold6_mfcc7===
using resnet model: 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 30, 130, 64)  3200        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 30, 130, 64)  256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 30, 130, 64)  0           batch_normalization_53[0][0]     
________________________

Epoch 1/3000
52/52 [==============================] - 16s 302ms/step - loss: 9.0078 - acc: 0.0604 - val_loss: 8.8012 - val_acc: 0.0647

Epoch 00001: val_acc improved from -inf to 0.06469, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 2/3000
52/52 [==============================] - 9s 165ms/step - loss: 7.7923 - acc: 0.0739 - val_loss: 7.1946 - val_acc: 0.0997

Epoch 00002: val_acc improved from 0.06469 to 0.09973, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 3/3000
52/52 [==============================] - 9s 165ms/step - loss: 6.8076 - acc: 0.0995 - val_loss: 6.8510 - val_acc: 0.0782

Epoch 00003: val_acc did not improve from 0.09973
Epoch 4/3000
52/52 [==============================] - 9s 166ms/step - loss: 6.0597 - acc: 0.1187 - val_loss: 6.1507 - val_acc: 0.0485

Epoch 00004: val_acc did not improve from 0.09973
Epoch 5/3000
52/52 [==============================] - 9s 166ms/step - loss: 5.5071 - acc: 0.1385 - val_loss: 5.0723 - val_acc: 0.1725

Epoch 00005: val_

52/52 [==============================] - 9s 167ms/step - loss: 2.7563 - acc: 0.4567 - val_loss: 2.3626 - val_acc: 0.4286

Epoch 00042: val_acc did not improve from 0.48248
Epoch 43/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7926 - acc: 0.4555 - val_loss: 2.2227 - val_acc: 0.4394

Epoch 00043: val_acc did not improve from 0.48248
Epoch 44/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7858 - acc: 0.4444 - val_loss: 2.5051 - val_acc: 0.3747

Epoch 00044: val_acc did not improve from 0.48248
Epoch 45/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7206 - acc: 0.4627 - val_loss: 2.0551 - val_acc: 0.5229

Epoch 00045: val_acc improved from 0.48248 to 0.52291, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 46/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7352 - acc: 0.4769 - val_loss: 2.2166 - val_acc: 0.5013

Epoch 00046: val_acc did not improve from 0.52291
Epoch 47/3000
52/52 [========

52/52 [==============================] - 9s 167ms/step - loss: 2.4187 - acc: 0.5691 - val_loss: 1.6502 - val_acc: 0.6334

Epoch 00084: val_acc improved from 0.61995 to 0.63342, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 85/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4170 - acc: 0.5673 - val_loss: 1.7660 - val_acc: 0.5633

Epoch 00085: val_acc did not improve from 0.63342
Epoch 86/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4438 - acc: 0.5634 - val_loss: 1.8612 - val_acc: 0.5580

Epoch 00086: val_acc did not improve from 0.63342
Epoch 87/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4751 - acc: 0.5493 - val_loss: 2.3727 - val_acc: 0.4232

Epoch 00087: val_acc did not improve from 0.63342
Epoch 88/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4273 - acc: 0.5628 - val_loss: 1.9449 - val_acc: 0.5391

Epoch 00088: val_acc did not improve from 0.63342
Epoch 89/3000
52/52 [========

...

52/52 [==============================] - 9s 167ms/step - loss: 2.1699 - acc: 0.6698 - val_loss: 1.4664 - val_acc: 0.6873

Epoch 00169: val_acc did not improve from 0.73854
Epoch 170/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.1680 - acc: 0.6731 - val_loss: 1.6261 - val_acc: 0.6550

Epoch 00170: val_acc did not improve from 0.73854
Epoch 171/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.1360 - acc: 0.6863 - val_loss: 1.4699 - val_acc: 0.6631

Epoch 00171: val_acc did not improve from 0.73854
Epoch 172/3000
52/52 [==============================] - 9s 166ms/step - loss: 2.1501 - acc: 0.6728 - val_loss: 1.6077 - val_acc: 0.6334

Epoch 00172: val_acc did not improve from 0.73854
Epoch 173/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.1486 - acc: 0.6695 - val_loss: 1.5463 - val_acc: 0.6469

Epoch 00173: val_acc did not improve from 0.73854
Epoch 174/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.

52/52 [==============================] - 9s 167ms/step - loss: 2.0982 - acc: 0.6767 - val_loss: 1.8036 - val_acc: 0.5984

Epoch 00213: val_acc did not improve from 0.73854
Epoch 214/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.0587 - acc: 0.6944 - val_loss: 1.3289 - val_acc: 0.7116

Epoch 00214: val_acc did not improve from 0.73854
Epoch 215/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.0285 - acc: 0.7097 - val_loss: 1.6219 - val_acc: 0.6253

Epoch 00215: val_acc did not improve from 0.73854
Epoch 216/3000
52/52 [==============================] - 9s 166ms/step - loss: 2.0809 - acc: 0.6845 - val_loss: 1.4737 - val_acc: 0.6846

Epoch 00216: val_acc did not improve from 0.73854
Epoch 217/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.0758 - acc: 0.6974 - val_loss: 1.7397 - val_acc: 0.6253

Epoch 00217: val_acc did not improve from 0.73854
Epoch 218/3000
52/52 [==============================] - 9s 166ms/step - loss: 2.


Epoch 00256: val_acc did not improve from 0.75472
Epoch 257/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9667 - acc: 0.7275 - val_loss: 1.6442 - val_acc: 0.5876

Epoch 00257: val_acc did not improve from 0.75472
Epoch 258/3000
52/52 [==============================] - 9s 168ms/step - loss: 1.9630 - acc: 0.7254 - val_loss: 1.5204 - val_acc: 0.6361

Epoch 00258: val_acc did not improve from 0.75472
Epoch 259/3000
52/52 [==============================] - 9s 166ms/step - loss: 1.9764 - acc: 0.7287 - val_loss: 1.3257 - val_acc: 0.7170

Epoch 00259: val_acc did not improve from 0.75472
Epoch 260/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.0032 - acc: 0.7130 - val_loss: 1.5825 - val_acc: 0.6388

Epoch 00260: val_acc did not improve from 0.75472
Epoch 261/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9750 - acc: 0.7290 - val_loss: 1.5924 - val_acc: 0.6631

Epoch 00261: val_acc did not improve from 0.75472
Epoch 262/3000


Epoch 00300: val_acc did not improve from 0.75472
Epoch 301/3000
52/52 [==============================] - 9s 166ms/step - loss: 1.9486 - acc: 0.7476 - val_loss: 1.4373 - val_acc: 0.6819

Epoch 00301: val_acc did not improve from 0.75472
Epoch 302/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9343 - acc: 0.7437 - val_loss: 1.2516 - val_acc: 0.7385

Epoch 00302: val_acc did not improve from 0.75472
Epoch 303/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9653 - acc: 0.7251 - val_loss: 1.2921 - val_acc: 0.7412

Epoch 00303: val_acc did not improve from 0.75472
Epoch 304/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9360 - acc: 0.7365 - val_loss: 1.4472 - val_acc: 0.6765

Epoch 00304: val_acc did not improve from 0.75472
Epoch 305/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9468 - acc: 0.7299 - val_loss: 1.1882 - val_acc: 0.7439

Epoch 00305: val_acc did not improve from 0.75472
Epoch 306/3000

52/52 [==============================] - 9s 167ms/step - loss: 1.9225 - acc: 0.7527 - val_loss: 1.5387 - val_acc: 0.6604

Epoch 00344: val_acc did not improve from 0.78437
Epoch 345/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.8721 - acc: 0.7635 - val_loss: 1.1735 - val_acc: 0.7466

Epoch 00345: val_acc did not improve from 0.78437
Epoch 346/3000
52/52 [==============================] - 9s 166ms/step - loss: 1.9062 - acc: 0.7527 - val_loss: 1.5853 - val_acc: 0.6685

Epoch 00346: val_acc did not improve from 0.78437
Epoch 347/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.9140 - acc: 0.7533 - val_loss: 1.5410 - val_acc: 0.6658

Epoch 00347: val_acc did not improve from 0.78437
Epoch 348/3000
52/52 [==============================] - 9s 167ms/step - loss: 1.8747 - acc: 0.7566 - val_loss: 1.4062 - val_acc: 0.7008

Epoch 00348: val_acc did not improve from 0.78437
Epoch 349/3000
52/52 [==============================] - 9s 166ms/step - loss: 1.

In [ ]:
#MFCC6
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=> 0.85175(resnet3_semi)、0.82749(not semi)
#6 => 0.85904(resnet4_semi)、0.79784(not semi)
# 7=>0.88949(resnet2_semi)、0.83558(not semi)
# 8=>0.83558(resnet1 not semi)
# 9=>0.86792(resnet2_semi)、0.8112(not semi)